### ----------- GAIA matching function     ---------
### PURPOSE: Find match of stars with GAIA
### INPUT:   1. list of x,y positions in pixels
###          2. header of astrometry image to get WCS (need to know the coordinates ra,dec)
###          3. ra and dec of center image
### OPTIONAL INPUT:
###          - width: width of image in deg (may be slightly larger)
###          - height: height of image in deg (may be larger)
### OUTPUT:  centroid (ypix,xpix) of matched stars
### DEPENDENCIES: photutils,astropy (python)

In [ ]:
def match_gaia(sources,header,ra,dec,width=0.2,height=0.2):

    from astropy.coordinates import SkyCoord
    from astroquery.gaia import Gaia
    from astropy.wcs import WCS
    from astropy import units as u
    Gaia.MAIN_GAIA_TABLE = "gaiaedr3.gaia_source"#edr3 or dr2
    Gaia.ROW_LIMIT = -1

    ## get ra,dec from WCS astrometry header
    wcs_header = WCS(header)
    #coords = wcs_header.pixel_to_world(sources['xcentroid'],sources['ycentroid'])
    coords = wcs_header.pixel_to_world(sources[:,0],sources[:,1])


    ## get Gaia catalog around center ra/dec values
    cencoord = SkyCoord(ra=ra,dec=dec,unit=(u.deg,u.deg),frame='icrs')
    width,height = u.Quantity(width, u.deg),u.Quantity(height, u.deg)
    gaia_stars = Gaia.query_object_async(coordinate=cencoord, width=width, height=height)
    gaia_coords = SkyCoord(ra=gaia_stars['ra'],dec=gaia_stars['dec'])

    ## match catalogs
    gidx, gd2d, gd3d = coords.match_to_catalog_sky(gaia_coords)
    gbestidx=(gd2d.deg < 0.0008)                         #<0.00015deg=0.54''

    ## output variables
    star_ra,star_dec = np.zeros(len(sources),dtype=float),np.zeros(len(sources),dtype=float)
    star_ra[:],star_dec[:] = np.nan,np.nan
    star_ra[gbestidx] = gaia_stars['ra'][gidx[gbestidx]]
    star_dec[gbestidx] = gaia_stars['dec'][gidx[gbestidx]]
    star_par,star_parer = np.zeros(len(sources),dtype=float)*np.nan,np.zeros(len(sources),dtype=float)*np.nan
    star_par[gbestidx] = gaia_stars['parallax'][gidx[gbestidx]]
    star_parer[gbestidx] = gaia_stars['parallax_error'][gidx[gbestidx]]

    ##print ds9 region
    #f=open('/home/santiago/crisp/CAFOS-POL/20170713/2017eaw/regiontest.reg','w')
    #for gst in gaia_stars: f.write("icrs;circle(%f,%f,0.0005)\n" %(gst['ra'],gst['dec']))
    #f.close()
    #f=open('/home/santiago/crisp/CAFOS-POL/20170713/2017eaw/regiontest2.reg','w')
    #f.write('global color=red\n')
    #for co in coords: f.write("icrs;circle(%f,%f,0.001)\n" %(co.ra.value,co.dec.value))
    #f.close()
    return star_ra,star_dec,star_par,star_parer
